In [1]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
       
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

def plot_vg_images(vg_images):
    plt.figure(figsize=(10, 10))
    num_images = len(vg_images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(vg_images[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def combine_data_from_folder(folder_path , batch_size=100):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp


folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\processed_npz_files_VitalDB_Final'

vg_image_size = 224 


combined_data_generator = combine_data_from_folder(folder_path,224)

# FOR SINGLE BATCH

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

vg_image_dir = 'vg_images'


def load_vg_images(batch_idx):
    vg_images_path = os.path.join(vg_image_dir, f'vg_images_batch_{batch_idx}.npy')
    inverted_vg_images_path = os.path.join(vg_image_dir, f'inverted_vg_images_batch_{batch_idx}.npy')
    
    if not os.path.exists(vg_images_path) or not os.path.exists(inverted_vg_images_path):
        print(f"VG image files do not exist for Batch {batch_idx}.")
        return None, None
    
    vg_images = np.load(vg_images_path)
    inverted_vg_images = np.load(inverted_vg_images_path)
    
    return vg_images, inverted_vg_images

In [ ]:
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

   
    
    vg_images, inverted_vg_images = load_vg_images(batch_idx+1)

    print("Flattening VG images for t-SNE...")
    
    flattened_vg_images = vg_images
    
    print("Combining SBP and DBP values...")
    
    y = np.column_stack((combined_seg_sbp_batch.flatten(), combined_seg_dbp_batch.flatten()))  
    
    break
    

In [ ]:
print("Applying PCA...")
    
pca = PCA(n_components=3)
X_pca = pca.fit_transform(flattened_vg_images)

print("Applying t-SNE...")
    
tsne = TSNE(n_components=3, perplexity=30)
X_tsne = tsne.fit_transform(X_pca)

In [ ]:
print("Plotting t-SNE results...")
    
normal_bp_condition = (q[:, 0] <= 120) | (q[:, 1] <= 80)  
high_bp_condition = ~normal_bp_condition  
plt.figure(figsize=(12, 8))
plt.scatter(P_tsne[normal_bp_condition, 0], P_tsne[normal_bp_condition, 1], 
        c='blue', marker='x', label='Normal BP')
plt.scatter(P_tsne[high_bp_condition, 0], P_tsne[high_bp_condition, 1], 
        c='red', marker='o', label='High BP')
plt.title('t-SNE Visualization of VG Images for PPG Signals')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
plt.show()

print(f"Batch {batch_idx + 1} processing complete.")
 

In [ ]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1)

ridge_model.fit(X_pca,y)

ridge_model.predict(X_pca)


In [ ]:
print(y)

# FOR MULTIPLE BATCHES

In [2]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import os
import matplotlib.pyplot as plt

vg_image_dir = 'vg_images'

# Function to load VG images from a given batch index
def load_vg_images(batch_idx):
    vg_images_path = os.path.join(vg_image_dir, f'vg_images_batch_{batch_idx}.npy')
    inverted_vg_images_path = os.path.join(vg_image_dir, f'inverted_vg_images_batch_{batch_idx}.npy')
    
    if not os.path.exists(vg_images_path) or not os.path.exists(inverted_vg_images_path):
        print(f"VG image files do not exist for Batch {batch_idx}.")
        return None, None
    
    vg_images = np.load(vg_images_path)
    inverted_vg_images = np.load(inverted_vg_images_path)
    
    return vg_images, inverted_vg_images

# Initialize lists to accumulate VG images and labels
all_vg_images = []
all_invrt_images = []
all_sbp_values = []
all_dbp_values = []


for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

    
    vg_images, inverted_vg_images = load_vg_images(batch_idx + 1)
    if vg_images is None:
        print(f"No VG images found for Batch {batch_idx + 1}. Stopping...")
        break
    print("Flattening VG images for t-SNE...")
    
    flattened_vg_images = vg_images
    flattened_inverted_images= inverted_vg_images
    
    all_vg_images.append(flattened_vg_images)
    all_invrt_images.append(flattened_inverted_images)
    all_sbp_values.append(combined_seg_sbp_batch.flatten())
    all_dbp_values.append(combined_seg_dbp_batch.flatten())

    print(f"Batch {batch_idx + 1} processing complete.")

    

all_vg_images = np.concatenate(all_vg_images, axis=0)
all_invrt_images = np.concatenate(all_invrt_images, axis=0)
all_sbp_values = np.concatenate(all_sbp_values, axis=0)
all_dbp_values = np.concatenate(all_dbp_values, axis=0) 

Processing Batch 1...
Flattening VG images for t-SNE...
Batch 1 processing complete.
Processing Batch 2...
Flattening VG images for t-SNE...
Batch 2 processing complete.
Processing Batch 3...
Flattening VG images for t-SNE...
Batch 3 processing complete.
Processing Batch 4...
Flattening VG images for t-SNE...
Batch 4 processing complete.
Processing Batch 5...
Flattening VG images for t-SNE...
Batch 5 processing complete.
Processing Batch 6...
VG image files do not exist for Batch 6.
No VG images found for Batch 6. Stopping...


In [3]:
print(all_vg_images.shape)

(4093, 50176)


In [4]:
print("Combining SBP and DBP values...")
# Combine SBP and DBP as labels
q = np.column_stack((all_sbp_values, all_dbp_values))

Combining SBP and DBP values...


In [88]:
print("Applying PCA...")
# Apply PCA
pca = PCA(n_components=100)
P_pca = pca.fit_transform(all_vg_images)
PI_pca = pca.fit_transform(all_invrt_images)

print("Applying t-SNE...")
# Apply t-SNE
tsne = TSNE(n_components=3, perplexity=30)
P_tsne = tsne.fit_transform(P_pca)
PI_tsne = tsne.fit_transform(PI_pca)

Applying PCA...
Applying t-SNE...


In [89]:
P_tsne

array([[-2.3792953e+00, -8.7788308e-01,  8.0992355e+00],
       [ 2.4168339e+00, -1.3485951e+00, -7.7982998e+00],
       [ 7.1165934e+00, -2.7806994e-01, -1.0010098e+00],
       ...,
       [-5.1258993e+00, -2.3973907e+01,  4.7974463e+00],
       [ 2.1295749e-02, -2.2062672e+01,  6.3254538e+00],
       [ 1.8684341e+00, -2.2611248e+01,  6.7794847e+00]], dtype=float32)

In [90]:
P_pca

array([[-3.13788681, -5.8880652 ,  7.62067748, ...,  0.7385554 ,
         1.26696676,  1.22253864],
       [-0.5764386 ,  1.88581685, -7.25458396, ..., -0.38350507,
        -0.27085556, -1.13445302],
       [-0.22859395, -3.94169228,  7.08548245, ..., -1.25537313,
         0.34196876, -1.08170369],
       ...,
       [ 0.51069574, -0.96911281,  2.80151871, ..., -2.03117393,
         0.30171742, -1.93890308],
       [-0.33968036, -0.3775972 , -1.79553166, ..., -0.94866563,
        -0.52619603, -0.93286915],
       [ 0.13911528, -0.90006169, -1.12458728, ..., -0.62850539,
        -0.74351818,  0.06184352]])

# PLOTTING t-SNE FOR NORMAL VG

In [ ]:
print("Plotting t-SNE results...")
# Plot t-SNE results
normal_bp_condition = (q[:, 0] <= 120) | (q[:, 1] <= 80)  
high_bp_condition = ~normal_bp_condition  

plt.figure(figsize=(12, 8))

plt.scatter(P_tsne[normal_bp_condition, 0], P_tsne[normal_bp_condition, 1], 
            c='blue', marker='x', label='Normal BP')
plt.scatter(P_tsne[high_bp_condition, 0], P_tsne[high_bp_condition, 1], 
            c='red', marker='o', label='High BP')
plt.title('t-SNE Visualization of VG Images for PPG Signals')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
plt.show()

# PLOTTING t-SNE FOR INVERTED VG

In [ ]:
print("Plotting t-SNE results...")
# Plot t-SNE results
normal_bp_condition = (q[:, 0] <= 120) | (q[:, 1] <= 80)  
high_bp_condition = ~normal_bp_condition  

plt.figure(figsize=(12, 8))

plt.scatter(PI_tsne[normal_bp_condition, 0], PI_tsne[normal_bp_condition, 1], 
            c='blue', marker='x', label='Normal BP')
plt.scatter(PI_tsne[high_bp_condition, 0], PI_tsne[high_bp_condition, 1], 
            c='red', marker='o', label='High BP')
plt.title('t-SNE Visualization of VG Images for PPG Signals')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
plt.show()

# RIDGE REGRESSION FOR NORMAL VG

In [91]:
from sklearn.model_selection import train_test_split

vg_train,vg_test,label_train,label_test=train_test_split(P_tsne,q, test_size=0.5, random_state=42)

In [97]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1)

ridge_model.fit(vg_train,label_train)

PREDICTED=ridge_model.predict(vg_test)


In [98]:
print(label_test)

[[111.42947482  74.20514106]
 [145.08559425  95.22235135]
 [124.89474619  74.19285634]
 ...
 [102.59097251  63.26930883]
 [111.58027454  74.97070195]
 [123.0635736   71.20892764]]


In [99]:
print(PREDICTED)

[[119.392265  74.78658 ]
 [120.24118   77.70854 ]
 [117.2084    69.42831 ]
 ...
 [115.70408   66.765434]
 [119.78525   75.55546 ]
 [116.54101   67.99354 ]]


In [100]:
predicted_sbp = PREDICTED[:, 0]  # Assuming the first column is SBP
predicted_dbp = PREDICTED[:, 1]  # Assuming the second column is DBP

true_sbp = label_test[:, 0]  # True SBP values from q
true_dbp = label_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.09408376307172961, MAE=13.251725218433041, RMSE=18.8319122658457, ME±SD=(-0.5943515648635542, 18.822530802361673)
DBP Metrics: R=0.2609031756589331, MAE=8.383734847143582, RMSE=12.926223805519282, ME±SD=(-0.11361014930053939, 12.925724529184013)


In [64]:
from sklearn.model_selection import train_test_split

vg_train,vg_test,label_train,label_test=train_test_split(all_vg_images,q, test_size=0.5, random_state=45)

In [65]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1)

ridge_model.fit(vg_train,label_train)

PREDICTED=ridge_model.predict(vg_test)


In [66]:
predicted_sbp = PREDICTED[:, 0]  # Assuming the first column is SBP
predicted_dbp = PREDICTED[:, 1]  # Assuming the second column is DBP

true_sbp = label_test[:, 0]  # True SBP values from q
true_dbp = label_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.4827649473119918, MAE=14.24032378290988, RMSE=18.758568463470958, ME±SD=(-0.8004565776150941, 18.741482333745104)
DBP Metrics: R=0.5024925985135684, MAE=9.743388554410146, RMSE=12.948562414171704, ME±SD=(-0.1901972129159548, 12.947165466382947)


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import BayesianRidge

# Create a multi-output regressor
multi_output_model = MultiOutputRegressor(BayesianRidge())

# Fit the model
multi_output_model.fit(P_pca, q)

# Predict
bayes_predict = multi_output_model.predict(P_pca)

# Extract predictions for SBP and DBP
sbp_predictions = bayes_predict[:, 0]
dbp_predictions = bayes_predict[:, 1]



In [ ]:
print(bayes_predict)

In [ ]:
predicted_sbp =bayes_predict[:, 0]  # Assuming the first column is SBP
predicted_dbp =bayes_predict[:, 1]  # Assuming the second column is DBP

true_sbp = q[:, 0]  # True SBP values from q
true_dbp = q[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

In [ ]:
bp_labels = []
for sbp, dbp in zip(predicted_sbp, predicted_dbp):
    if sbp > 125 or dbp > 85:
        bp_labels.append("high BP")
    else:
        bp_labels.append("normal BP")

# Count occurrences
normal_bp_count = bp_labels.count("normal BP")
high_bp_count = bp_labels.count("high BP")

print(f"Normal BP count: {normal_bp_count}")
print(f"High BP count: {high_bp_count}")

# RIDGE REGRESSION FOR INVERTED VG

In [ ]:
ridge_model.fit(P_pca,q)

PREDICTED_INVERTED=ridge_model.predict(PI_pca)

In [ ]:
predicted_sbp = PREDICTED_INVERTED[:, 0]  # Assuming the first column is SBP
predicted_dbp = PREDICTED_INVERTED[:, 1]  # Assuming the second column is DBP

true_sbp = q[:, 0]  # True SBP values from q
true_dbp = q[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

# VGG 19 FOR SINGLE BATCH

In [ ]:
import numpy as np
import os
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.linear_model import Ridge

# Load VG images (normal and inverted)
vg_image_dir = 'vg_images'
vg_images_normal = np.load(os.path.join(vg_image_dir, 'vg_images_batch_1.npy'))  # Adjust as needed
vg_images_inverted = np.load(os.path.join(vg_image_dir, 'inverted_vg_images_batch_1.npy'))  # Adjust as needed

# Reshape VG images for 3-channel input
vg_images_normal_reshaped = vg_images_normal.reshape(-1, 224, 224, 1)  # Add channel dimension
vg_images_inverted_reshaped = vg_images_inverted.reshape(-1, 224, 224, 1)

# Convert to 3 channels
vg_images_normal_rgb = np.concatenate([vg_images_normal_reshaped, vg_images_normal_reshaped, vg_images_normal_reshaped], axis=-1)
vg_images_inverted_rgb = np.concatenate([vg_images_inverted_reshaped, vg_images_inverted_reshaped, vg_images_inverted_reshaped], axis=-1)

# Preprocess images
vg_images_normal_preprocessed = preprocess_input(vg_images_normal_rgb)
vg_images_inverted_preprocessed = preprocess_input(vg_images_inverted_rgb)


In [ ]:
# Load VGG19 model without the top classifier layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a feature extraction model
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)


In [ ]:
# Extract features
vg_features_normal = feature_extractor.predict(vg_images_normal_preprocessed)
vg_features_inverted = feature_extractor.predict(vg_images_inverted_preprocessed)

# Flatten the features
vg_features_normal_flattened = vg_features_normal.reshape(vg_features_normal.shape[0], -1)
vg_features_inverted_flattened = vg_features_inverted.reshape(vg_features_inverted.shape[0], -1)

# Combine features
vg_features_combined = np.hstack((vg_features_normal_flattened, vg_features_inverted_flattened))  # Shape: (N, P)


In [ ]:
# Fit Ridge Regression
ridge_model = Ridge(alpha=1.0)  # Adjust regularization parameter as needed
ridge_model.fit(vg_features_combined, y)

# Predict SBP and DBP values
predicted_values = ridge_model.predict(vg_features_combined)

print(predicted_values)

In [ ]:
print(y)

In [ ]:
predicted_sbp = predicted_values[:, 0]  # Assuming the first column is SBP
predicted_dbp = predicted_values[:, 1]  # Assuming the second column is DBP

true_sbp = y[:, 0]  # True SBP values from q
true_dbp = y[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

# VGG19 FOR MULTIPLE BATCHES

In [5]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge


vg_images_normal = all_vg_images
vg_images_inverted = all_invrt_images

In [6]:
batch_size = 32

def image_generator(images, batch_size):
    num_samples = images.shape[0]
    for offset in range(0, num_samples, batch_size):
        batch_images = images[offset:offset+batch_size]
        batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
        batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
        batch_images_preprocessed = preprocess_input(batch_images_rgb)
        yield batch_images_preprocessed


base_model = VGG19(weights='imagenet', include_top=True, input_shape=(224, 224, 3))


#feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

def extract_features(generator, num_samples):
    features = []
    for batch in generator:
        batch_features = feature_extractor.predict(batch)
        features.append(batch_features)
    return np.vstack(features)

In [8]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
feature_extractor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
num_samples_normal = vg_images_normal.shape[0]
num_samples_inverted = vg_images_inverted.shape[0]


normal_generator = image_generator(vg_images_normal, batch_size)
inverted_generator = image_generator(vg_images_inverted, batch_size)


vg_features_normal = extract_features(normal_generator, num_samples_normal)
vg_features_inverted = extract_features(inverted_generator, num_samples_inverted)


vg_features_normal_flattened = vg_features_normal.reshape(vg_features_normal.shape[0], -1)
vg_features_inverted_flattened = vg_features_inverted.reshape(vg_features_inverted.shape[0], -1)


vg_features_combined = np.hstack((vg_features_normal_flattened, vg_features_inverted_flattened))  # Shape: (N, P)

1/1 [==============================] - 4s 4s/step


In [16]:
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_combined, q)


predicted_values = ridge_model.predict(vg_features_combined)

print(predicted_values)

[[116.43793   76.99057 ]
 [122.60095   75.38452 ]
 [114.535736  70.599976]
 ...
 [115.66052   71.652954]
 [127.835846  76.837494]
 [116.53198   66.91861 ]]


In [17]:
predicted_sbp =predicted_values[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values[:, 1]  # Assuming the second column is DBP

true_sbp = q[:, 0]  # True SBP values from q
true_dbp = q[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.772797072279392, MAE=9.155812839188988, RMSE=12.157537586010443, ME±SD=(0.0005058169353249202, 12.157537575488133)
DBP Metrics: R=0.790184526373008, MAE=5.9226458632022005, RMSE=8.282380747120175, ME±SD=(-0.00013118982836539695, 8.282380746081177)


In [10]:
feature_extractor2 = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)
feature_extractor2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [12]:
def extract_features2(generator, num_samples):
    features = []
    for batch in generator:
        batch_features = feature_extractor2.predict(batch)
        features.append(batch_features)
    return np.vstack(features)

In [13]:
num_samples_normal = vg_images_normal.shape[0]
num_samples_inverted = vg_images_inverted.shape[0]


normal_generator = image_generator(vg_images_normal, batch_size)
inverted_generator = image_generator(vg_images_inverted, batch_size)


vg_features_normal2 = extract_features2(normal_generator, num_samples_normal)
vg_features_inverted2 = extract_features2(inverted_generator, num_samples_inverted)


vg_features_normal_flattened2 = vg_features_normal2.reshape(vg_features_normal2.shape[0], -1)
vg_features_inverted_flattened2 = vg_features_inverted2.reshape(vg_features_inverted2.shape[0], -1)


vg_features_combined2 = np.hstack((vg_features_normal_flattened2, vg_features_inverted_flattened2))  # Shape: (N, P)

1/1 [==============================] - 5s 5s/step


In [14]:
from sklearn.model_selection import train_test_split

vg_features_train,vg_features_test,label_vgg_train,label_vgg_test=train_test_split(vg_features_combined2,q, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_train, label_vgg_train)


predicted_values_split = ridge_model.predict(vg_features_test)

In [15]:
predicted_sbp =predicted_values_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg_test[:, 0]  # True SBP values from q
true_dbp = label_vgg_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.7025930805917515, MAE=10.249272953918561, RMSE=13.327511856126822, ME±SD=(-0.680277537550576, 13.310138795185614)
DBP Metrics: R=0.7228155490015689, MAE=6.570788409793689, RMSE=9.177056106088898, ME±SD=(-0.3094514739362396, 9.171837251040941)


In [5]:
import numpy as np
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# Define the batch size
batch_size = 32

# Function to generate image batches
def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels

In [4]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

# Load the VGG19 model with pre-trained ImageNet weights and without the top layer
base_model = VGG19(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
#x = Dense(4096, activation='relu', name='fc1')(x)
#x = Dense(4096, activation='relu', name='fc2')(x)
output = Dense(2, activation='linear')(x)  # Adjust the output layer according to your task

# Create a new model with the modified top layer
model = Model(inputs=base_model.input, outputs=output)

# Print a summary to verify the layers are frozen
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [7]:
# Load the base VGG19 model with the top layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
output = Dense(2, activation='linear')(x)  # Adjust the output layer according to your task

# Create a new model with the modified top layer
model = Model(inputs=base_model.input, outputs=output)

In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [9]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_error', metrics=['mae'])

# Assuming you have images and labels loaded
images = vg_images_normal
labels = q

# Create the image generator
train_generator = image_generator(images, labels, batch_size)

# Train the model
num_samples = images.shape[0]
model.fit(train_generator, steps_per_epoch=num_samples // batch_size, epochs=1)

C:\Users\nihal\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


127/127 [==============================] - 2638s 21s/step - loss: 984.7535 - mae: 21.3258


In [13]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)

# Create the generator for feature extraction (no labels needed)
feature_generator_normal = image_generator(images, np.zeros((num_samples,)), batch_size)
feature_generator_inverted = image_generator(vg_images_inverted, np.zeros((num_samples,)), batch_size)

# Extract features
features_normal = extract_features(feature_generator_normal, num_samples)
features_inverted = extract_features(feature_generator_inverted, num_samples)

features_normal_flattened= features_normal.reshape(features_normal.shape[0],-1)
features_inverted_flattened= features_inverted.reshape(features_inverted.shape[0],-1)

features= np.hstack((features_normal_flattened,features_inverted_flattened))

1/1 [==============================] - 6s 6s/step


In [14]:
from sklearn.model_selection import train_test_split

vg_features_train,vg_features_test,label_vgg_train,label_vgg_test=train_test_split(features,q, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_train, label_vgg_train)


predicted_values_split = ridge_model.predict(vg_features_test)

In [15]:
predicted_sbp =predicted_values_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg_test[:, 0]  # True SBP values from q
true_dbp = label_vgg_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.2004484189452353, MAE=13.616361949466357, RMSE=18.24446058044627, ME±SD=(-1.399250513359489, 18.190724006270916)
DBP Metrics: R=0.1983167134292075, MAE=9.396296807851911, RMSE=12.998157420654644, ME±SD=(-0.9204670452789594, 12.965524931551164)


In [8]:
orgininal_Mean_sbp=np.mean(q[:,0])
orgininal_Mean_dbp=np.mean(q[:,1])

In [11]:
original_SD_sbp=np.std(orgininal_Mean_sbp-q[:,0])
original_SD_dbp=np.std(orgininal_Mean_dbp-q[:,1])

In [17]:
print('original standard deviation of sbp:',original_SD_sbp)
print('original standard deviation of dbp:',original_SD_dbp)

original standard deviation of sbp: 19.092041821560084
original standard deviation of dbp: 13.45258445160312


In [17]:
from sklearn.model_selection import train_test_split

vg_features_train,vg_features_test,label_vgg_train,label_vgg_test=train_test_split(vg_features_combined,q, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_train, label_vgg_train)


predicted_values_split = ridge_model.predict(vg_features_test)

In [18]:
predicted_sbp =predicted_values_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg_test[:, 0]  # True SBP values from q
true_dbp = label_vgg_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.7393615884079727, MAE=9.680599027440008, RMSE=12.56047092306476, ME±SD=(-0.5797904002439807, 12.547082246520114)
DBP Metrics: R=0.7679653162002857, MAE=6.177144039920629, RMSE=8.488757550977363, ME±SD=(-0.27434023643242406, 8.484323319743856)


In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import numpy as np
from torchvision.models import vgg19
from sklearn.linear_model import Ridge

# Define a batch size
batch_size = 16  # Use a smaller batch size to manage memory

# Define a function to preprocess VG images
def preprocess_vg_images(vg_images, target_size=(224, 224)):
    print(f"Original VG images shape: {vg_images.shape}")
    vg_images_extended = vg_images.reshape(-1, 224, 224)  # Ensure images are reshaped correctly
    print(f"Extended VG images shape: {vg_images_extended.shape}")
    
    resize_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(target_size),
        transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
        transforms.ToTensor()
    ])
    
    vg_images_resized = []
    for image in vg_images_extended:
        transformed_image = resize_transform(image)
        vg_images_resized.append(transformed_image.numpy())
    
    vg_images_resized = np.array(vg_images_resized)
    print(f"Resized VG images shape: {vg_images_resized.shape}")
    return vg_images_resized

# Load VG images and inverted VG images (replace this with actual loading code)
vg_images = all_vg_images
inverted_vg_images = all_invrt_images

# Preprocess VG images and inverted VG images
vg_images_resized = preprocess_vg_images(vg_images)
inverted_vg_images_resized = preprocess_vg_images(inverted_vg_images)

# Convert to PyTorch tensors
vg_images_tensor = torch.tensor(vg_images_resized, dtype=torch.float32)
inverted_vg_images_tensor = torch.tensor(inverted_vg_images_resized, dtype=torch.float32)

# Create TensorDatasets
vg_dataset = TensorDataset(vg_images_tensor)
inverted_vg_dataset = TensorDataset(inverted_vg_images_tensor)

# Create DataLoaders
vg_loader = DataLoader(vg_dataset, batch_size=batch_size, shuffle=False)
inverted_vg_loader = DataLoader(inverted_vg_dataset, batch_size=batch_size, shuffle=False)

# Load VGG19 model and remove the top layers
model = vgg19(weights='VGG19_Weights.DEFAULT')
model = torch.nn.Sequential(*list(model.children())[:-1])

# Ensure the model is in evaluation mode
model.eval()

# Define a function to extract features in batches
def extract_features(loader):
    features = []
    with torch.no_grad():
        for batch in loader:
            batch_images = batch[0]
            batch_features = model(batch_images)
            features.append(batch_features.squeeze().numpy())
    return np.vstack(features)

# Extract features from VG images and inverted VG images in batches
vg_features = extract_features(vg_loader)
inverted_vg_features = extract_features(inverted_vg_loader)

print(f"VG features shape: {vg_features.shape}")
print(f"Inverted VG features shape: {inverted_vg_features.shape}")

# Flatten the features
vg_features_flattened = vg_features.reshape(vg_features.shape[0], -1)
inverted_vg_features_flattened = inverted_vg_features.reshape(inverted_vg_features.shape[0], -1)

# Combine VG features and inverted VG features
combined_features = np.hstack((vg_features_flattened, inverted_vg_features_flattened))
print(f"Combined features shape: {combined_features.shape}")

# Fit a Ridge regression model
ridge = Ridge(alpha=1.0)
ridge.fit(combined_features, q)

# Predict SBP and DBP values
predictions = ridge.predict(combined_features)
print(predictions)


Original VG images shape: (4093, 50176)
Extended VG images shape: (4093, 224, 224)
Resized VG images shape: (4093, 3, 224, 224)
Original VG images shape: (4093, 50176)
Extended VG images shape: (4093, 224, 224)
Resized VG images shape: (4093, 3, 224, 224)
VG features shape: (4093, 512, 7, 7)
Inverted VG features shape: (4093, 512, 7, 7)
Combined features shape: (4093, 50176)
[[117.3998    77.530785]
 [115.71025   70.64775 ]
 [116.23215   71.17795 ]
 ...
 [141.7895    81.69441 ]
 [146.8068    84.19699 ]
 [146.71292   85.36191 ]]


In [8]:
predicted_sbp =predictions[:, 0]  # Assuming the first column is SBP
predicted_dbp =predictions[:, 1]  # Assuming the second column is DBP

true_sbp = q[:, 0]  # True SBP values from q
true_dbp = q[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.9994690989845076, MAE=0.4871802901805776, RMSE=0.6426835897578517, ME±SD=(-0.00015217686876435428, 0.6426835717413657)
DBP Metrics: R=0.9994915580394712, MAE=0.3326262971835257, RMSE=0.4432602140310017, ME±SD=(8.402787939906152e-05, 0.44326020606650995)


In [9]:
from sklearn.model_selection import train_test_split

vg_features2_train,vg_features2_test,label_vgg2_train,label_vgg2_test=train_test_split(combined_features,q, test_size=0.5, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features2_train, label_vgg2_train)


predicted_values2_split = ridge_model.predict(vg_features2_test)

In [10]:
predicted_sbp =predicted_values2_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values2_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg2_test[:, 0]  # True SBP values from q
true_dbp = label_vgg2_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.6905491016342521, MAE=11.004855136935634, RMSE=14.448826650120834, ME±SD=(0.3414734326884799, 14.444791014757186)
DBP Metrics: R=0.7078827862102354, MAE=7.6090561932947605, RMSE=10.072771913413334, ME±SD=(0.35926332227121105, 10.066362991911188)
